In [ ]:
import requests
import json

In [ ]:
datawolf_base = "https://incore-dev.ncsa.illinois.edu/datawolf/"
datwolf_workflow_uri = datawolf_base + "workflows/"

In [ ]:
def get_workflow_def(datwolf_workflow_uri, workflow_id,token):
    res = requests.get(datwolf_workflow_uri+workflow_id , headers={"authorization": token})
    workflow_def = res.json()
    
    return workflow_def

In [ ]:
# template
def construct_datwolf_config_template(workflow_def):
    return {
        "title": workflow_def["title"],
        "description": workflow_def["description"],
        "wfTitle": workflow_def["title"],
        "wfId": workflow_def["id"],
        "creatorId": workflow_def["creator"]["id"], 
        "inputs": {
            "parameters": [],
            "datasets": []
        },
        "outputs": {
            "datasets":[{
                "title": "",
                "description": "",
                "datasetsMap": [],
            }]
        },
        "metadata":[]
    }

In [ ]:
def construct_datwolf_config_parameters(workflow_def, datawolf_config_template):
    parameters = {}
    for step in workflow_def["steps"]:
        for param in step["tool"]["parameters"]:
            if param["id"] not in parameters.keys():
                parameters[param["id"]] = {
                    "title": param["title"],
                    "description": param["description"],
                    "variableName":"",
                    "widget":None,
                    "hidden":True,
                    "defaultValue":param["value"],
                    "parametersMap":[{
                        "paramId": step["parameters"][param["parameterId"]],
                        "parameters": {
                            "step": step["title"],
                            "title": param["title"]
                        }
                    }]
                }
            else:
                # append to parametersMap
                parameters[param["id"]]["parametersMap"].append({
                    "paramId": step["parameters"][param["parameterId"]],
                        "parameters": {
                            "step": step["title"],
                            "title": param["title"]
                        }
                })
    for entry in parameters.values():
        datawolf_config["inputs"]["parameters"].append(entry)
    
    return datawolf_config_template

In [ ]:
def construct_datawolf_config_input_datasets(workflow_def, datawolf_config_template):
    datasets = {}
    for step in workflow_def["steps"]:
        for dt in step["tool"]["inputs"]:
            if dt["id"] not in datasets.keys():
                datasets[dt["id"]] = {
                        "title": dt["title"],
                        "description": dt["description"],
                        "variableName":"",
                        "widget":None,
                        "hidden":True,
                        "datasetsMap":[
                            {
                                "datasetId": step["inputs"][dt["dataId"]],
                                "datasets": {
                                    "step": step["title"],
                                    "title": dt["title"]
                            }
                        }]
                    }
            
            else:
                # append to parametersMap
                datasets[dt["id"]]["datasetsMap"].append({
                                "datasetId": step["inputs"][dt["dataId"]],
                                "datasets": {
                                    "step": step["title"],
                                    "title": dt["title"]
                                }
                            })
    for entry in datasets.values():
        datawolf_config["inputs"]["datasets"].append(entry)
    
    return datawolf_config_template

In [ ]:
def construct_datawolf_config_output_datasets(workflow_def, datawolf_config_template):
    
    # outputs are unique
    for step in workflow_def["steps"]:
        for dt in step["tool"]["outputs"]:
            if dt["title"] == "stdout":
                widget = "Log"
                title = step["title"]
                variableName = (step["title"]+ "Log").replace(" ", "").replace("_", "").replace("-", "")
            else:
                widget = ""
                title = dt["title"]
                variableName = ""
            datawolf_config["outputs"]["datasets"][0]["datasetsMap"].append({
                "datasetId":step["outputs"][dt["dataId"]],
                "datasets": {
                        "step": step["title"],
                        "title": title,
                        "variableName": variableName,
                        "widget": widget
                    }
            })     
    
    return datawolf_config_template

In [ ]:
workflow_id = "c81ab517-77a3-472a-a1a6-db08d1fadb35"
token = "bearer "
workflow_def = get_workflow_def(datwolf_workflow_uri, workflow_id,token)
datawolf_config = construct_datwolf_config_template(workflow_def)
datawolf_config = construct_datwolf_config_parameters(workflow_def, datawolf_config)
datawolf_config = construct_datawolf_config_input_datasets(workflow_def, datawolf_config)
datawolf_config = construct_datawolf_config_output_datasets(workflow_def, datawolf_config)

In [ ]:
# save as json
with open("datawolf.config.json", "w") as f:
    json.dump(datawolf_config, f, indent=2)